<a href="https://colab.research.google.com/github/SamManuJacob/BIS_lab_1BM23CS291/blob/main/Parallel_Cellular.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np

# Objective function (Rastrigin)
def f(x):
    return 10 * len(x) + np.sum(x**2 - 10 * np.cos(2 * np.pi * x))


# Moore neighborhood with input 3/5/7/9...
def get_neighbors_indices(i, j, num_rows, num_cols, neigh_size=3):
    """
    neigh_size must be odd: 3 → 3x3, 5 → 5x5, 7 → 7x7 Moore neighborhood.
    """
    if neigh_size % 2 == 0 or neigh_size < 3:
        raise ValueError("neigh_size must be an odd integer >= 3")

    radius = (neigh_size - 1) // 2  # 3→1, 5→2, 7→3

    neighbors = []
    for dx in range(-radius, radius + 1):
        for dy in range(-radius, radius + 1):

            if dx == 0 and dy == 0:
                continue  # skip center

            ni = (i + dx) % num_rows
            nj = (j + dy) % num_cols
            neighbors.append((ni, nj))

    return neighbors


# ------------------------
# PARAMETERS
# ------------------------
num_rows = 10
num_cols = 10
dim = 2
max_iter = 100
bounds = [-5.12, 5.12]

neigh_size = 5   # can be 3, 5, 7, 9...


# ------------------------
# Step 1: Initialize population
# ------------------------
grid = np.random.uniform(bounds[0], bounds[1], size=(num_rows, num_cols, dim))
fitness = np.zeros((num_rows, num_cols))

for i in range(num_rows):
    for j in range(num_cols):
        fitness[i, j] = f(grid[i, j])

# Best solution so far
best_pos = np.unravel_index(np.argmin(fitness), fitness.shape)
best_cell = grid[best_pos]
best_fitness = fitness[best_pos]


# ------------------------
# Step 2: Parallel Cellular Evolution
# ------------------------
for t in range(max_iter):

    new_grid = grid.copy()
    new_fitness = fitness.copy()

    for i in range(num_rows):
        for j in range(num_cols):

            # Neighborhood (custom size)
            neighbors_idx = get_neighbors_indices(
                i, j, num_rows, num_cols, neigh_size=neigh_size
            )

            neighbors = [grid[ni, nj] for ni, nj in neighbors_idx]
            neighbor_fitness = [fitness[ni, nj] for ni, nj in neighbors_idx]

            # Best neighbor
            best_neighbor = neighbors[np.argmin(neighbor_fitness)]

            # Movement toward best neighbor
            r = np.random.rand()
            xi_new = grid[i, j] + r * (best_neighbor - grid[i, j])

            # Boundary enforcement
            xi_new = np.clip(xi_new, bounds[0], bounds[1])

            # Evaluate
            f_new = f(xi_new)

            # Synchronous selection (update goes to new_grid only)
            if f_new < fitness[i, j]:
                new_grid[i, j] = xi_new
                new_fitness[i, j] = f_new

                # Global best
                if f_new < best_fitness:
                    best_fitness = f_new
                    best_cell = xi_new

    # Synchronous update
    grid = new_grid
    fitness = new_fitness


# ------------------------
# OUTPUT
# ------------------------
print("Best solution found:", best_cell)
print("Best fitness value:", best_fitness)


Best solution found: [4.93489576e-05 2.50086912e-06]
Best fitness value: 4.843889520600442e-07
